# HTML to dataframe
This notebook converts the City of Detroit Directory of Depts. contact website into a pandas data frame, merge it with the overall database then save it in a csv file

In [1]:
# Import
import pandas as pd
import numpy as np

In [2]:
df = pd.read_html('https://publish.smartsheet.com/9def816c9e6a4a4395d2903039bf714d') # convert the html table into pd data frame
df[0].drop(['Web Address', 'Department'], axis=1, inplace=True) # drop the two columns
df[0] = df[0].map(lambda x: np.nan if isinstance(x, str) and len(x) == 1 else x) # convert all one-character strings into np.nan
df[0] = df[0].loc[:, ~df[0].columns.str.contains('^Unnamed')] # drop the 'Unnamed: 0' column
df[0]['Name'] = df[0]['Name'].replace(['General Information','Administration'], np.nan) # convert unnecessary info in the name column to na
df[0][['Name', 'Title']] = df[0]['Name'].str.split('; ', n=1, expand=True) # split name and title to different columns
df[0].drop('Title', axis=1, inplace=True)
df[0] = df[0].fillna(value=np.nan) # fill all null value with np.nan
contact = df[0].dropna(how='all')[:-1] # drop row with all np.nan values and the last row which

In [3]:
names = pd.read_excel("List of names 1-7-2.xlsx") # Read in the List of provided names csv
names['Name'] = names['Last'] + ', ' + names['First'] # Make a new column 

merged_df = pd.merge(names, contact, on='Name', how="left") # Left join names and contact, all the rows in names will be preserved
merged_df['Phone'] = merged_df['Mobile'].combine_first(merged_df['Phone Number']) # Combine the two columns from the two dfs
merged_df['Email'] = merged_df['Email_x'].combine_first(merged_df['Email_y'])
merged_df.drop(columns=['Mobile', 'Phone Number', 'Email_x', 'Email_y', 'Fax'], inplace=True) # Drop all unnecessary columns
merged_df.dropna(subset=['Name'], inplace=True) # Drop all rows with no Name value
result = merged_df.drop('Name', axis=1) # Drop the Name column

In [4]:
result

,Pre,First,Middle,Last,Suffix,Title,Department/District,Board/Commission/Department,LinkedIn,Twitter,Location / Suite,Phone,Email
0,NaN,James,E,White,NaN,Chief of Police,Detroit Police Department (DPD),Executive Department,NaN,NaN,NaN,NaN,NaN
1,NaN,Charles,NaN,Sims,NaN,Fire Commissioner,Detroit Fire Department (DFD),Executive Department,NaN,NaN,NaN,NaN,NaN
2,NaN,Lawrence,NaN,Meyer,NaN,Director,Homeland Security & Emergency Management (HSEM),Executive Department,NaN,NaN,"13331 Lyndon Detroit, 48227",(313)596-2590,NaN
3,NaN,Antoine,NaN,Bryant,NaN,Director,Planning & Development Department (PDD),Executive Department,NaN,NaN,NaN,NaN,NaN
4,NaN,Dave,NaN,Bell,NaN,Director,"Buildings, Safety, Engineering, & Environmenta...",Executive Department,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,NaN,Sonya,NaN,Mays,NaN,Treasurer,NaN,Detroit Public Schools Community District,NaN,NaN,NaN,(313) 498-0629,sonya.mays@detroitk12.org
959,NaN,Sherry,NaN,Gay-Dagnogo,NaN,NaN,NaN,Detroit Public Schools Community District,NaN,NaN,NaN,(313) 683-3648,sonya.mays@detroitk12.org
960,NaN,Georgia,NaN,Lemmons,NaN,NaN,NaN,Detroit Public Schools Community District,NaN,NaN,NaN,(313) 657-0440,georgia.lemmons@detroitk12.org
961,Bishop,Corletta,J.,Vaughn,NaN,NaN,NaN,Detroit Public Schools Community District,NaN,NaN,NaN,(313) 539-9468,lamar.lemmons@detroitk12.org


In [5]:
result.to_csv('contact.csv', index=False) # convert df into csv file